In [2]:
import os 
import pandas as pd

data_csv="/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/experiments/tests_results_Surv.csv"
df= pd.read_csv(data_csv)
col = 'modality_setting'
modalities_to_check = ["complete","missing_all_60","missing_all_30","missing_all_wsi_60", "missing_all_genomics_60"]



In [ ]:
for mod in modalities_to_check:
    df_mod = df[df[col] == mod]
    if not df_mod.empty:
        
        df_mod=df_mod[df_mod["model_version"] =='Lowest_Validation_Loss_Model']
        df_mod = df_mod[['dataset_name','seed','c-index_mean', 'c-index_std']]
        print(df_mod['dataset_name'].value_counts())
        print(df_mod[df_mod['dataset_name']=='OV'])
        if df_mod['dataset_name'].value_counts()['OV'].item() > 3:
            df_mod=df_mod.iloc[:-3]
        summary = df_mod.groupby('dataset_name').agg(['mean']).reset_index()
        df_mod.to_csv(f'/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/OUR_MODEL_{mod}_results_OS_TCGA.csv', index=False)
        summary.to_csv(f'/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/OUR_MODEL_{mod}_results_OS_TCGA_grouped.csv', index=False)
    else:
        print(f"No results found for {mod}")

Results for complete:
dataset_name
OV              6
BLCA            3
COAD            3
BRCA            3
HNSC            3
KIRC            3
KIRP            3
LUAD            3
STAD            3
LUSC            3
NACT_primary    3
Name: count, dtype: int64
    dataset_name  seed  c-index_mean  c-index_std
78            OV  43.0      0.488891     0.079048
84            OV  45.0      0.491125     0.069606
87            OV  44.0      0.493371     0.072777
498           OV  44.0      0.569895     0.070766
507           OV  45.0      0.543654     0.097371
516           OV  43.0      0.572275     0.040349
Results for missing_all_60:
dataset_name
BLCA    6
HNSC    3
BRCA    3
LUAD    3
LUSC    3
STAD    3
COAD    3
KIRC    3
OV      3
KIRP    3
Name: count, dtype: int64
    dataset_name  seed  c-index_mean  c-index_std
276           OV  44.0      0.472056     0.015766
279           OV  43.0      0.477538     0.028944
288           OV  45.0      0.447711     0.029017
Results for missing_all_

In [5]:
import pandas as pd
from scipy.stats import wilcoxon, friedmanchisquare
from scipy.stats import ttest_rel




# Carica i due file CSV per i due modelli (ad esempio model_1.csv e model_2.csv)
df1 = pd.read_csv('/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/OUR_MODEL_complete_results_OS_TCGA.csv')
df2 = pd.read_csv('/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/HGCN_complete.csv')
df3 = pd.read_csv('/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/MOTCat_complete.csv')
df4= pd.read_csv('/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/survpath.csv')
model1 = 'custom_multimodal'
model2 = 'HGCN'
model3='MOTCat'
model4='Survpath'
filtered_df1 = df1[df1['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']]
filtered_df3 = df3[df3['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']].rename(columns={'c-index_mean': 'c-index_mean_model_3'})
filtered_df4 = df4[df4['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']].rename(columns={'c-index_mean': 'c-index_mean_model_4'})
df = pd.merge(filtered_df1,df2[['dataset_name', 'seed', 'c-index_mean']], on=['dataset_name', 'seed'],suffixes=('_model_1', '_model_2'))
df= pd.merge(df,
              filtered_df3[['dataset_name', 'seed', 'c-index_mean_model_3']],on=['dataset_name', 'seed'])
df= pd.merge(df,filtered_df4[['dataset_name', 'seed', 'c-index_mean_model_4']], on=['dataset_name', 'seed'])
# Lista dei tumori unici nel dataset
tumors = df['dataset_name'].unique()
ttest_p_values = {
    f"{model1}_vs_{model2}": {},
    f"{model1}_vs_{model3}": {},
    f"{model1}_vs_{model4}": {}
}
wilcoxon_p_values = {
    f"{model1}_vs_{model2}": {},
    f"{model1}_vs_{model3}": {},
    f"{model1}_vs_{model4}": {}
}
friedman_p_values = {}
# Itera attraverso ogni tumore
for tumor in tumors:
    print('Processing tumor:', tumor)
    # Filtra i dati per il tumore corrente
    tumor_data = df[df['dataset_name'] == tumor]
    
    # Assicurati di avere 3 valori di c-index per ogni seed
    if len(tumor_data) == 3:
        # Estrai i valori di c-index per i 2 modelli (model 1 e model 2) per i 3 seeds
        c_index_model_1 = tumor_data['c-index_mean_model_1'].values
        c_index_model_2 = tumor_data['c-index_mean_model_2'].values
        c_index_model_3 = tumor_data['c-index_mean_model_3'].values
        c_index_model_4 = tumor_data['c-index_mean_model_4'].values
        # Applica il test di Wilcoxon (test appaiato tra i 2 modelli)
        
        # Wilcoxon per ogni confronto con model_1
        wilcoxon_p_values[f"{model1}_vs_{model2}"][tumor] = wilcoxon(c_index_model_1, c_index_model_2).pvalue
        wilcoxon_p_values[f"{model1}_vs_{model3}"][tumor] = wilcoxon(c_index_model_1, c_index_model_3).pvalue
        wilcoxon_p_values[f"{model1}_vs_{model4}"][tumor] = wilcoxon(c_index_model_1, c_index_model_4).pvalue
        # c_index_model_1 e c_index_model_2 sono array con i C-index sui seed o pazienti
        # Applica il test t di Student (test appaiato tra i 2 modelli)
        ttest_p_values[f"{model1}_vs_{model2}"][tumor] = ttest_rel(c_index_model_1, c_index_model_2).pvalue
        ttest_p_values[f"{model1}_vs_{model3}"][tumor] = ttest_rel(c_index_model_1, c_index_model_3).pvalue
        ttest_p_values[f"{model1}_vs_{model4}"][tumor] = ttest_rel(c_index_model_1, c_index_model_4).pvalue
    
        # Applica il test di Friedman (per confrontare tutti i 3 seed)
        stat_friedman, p_value_friedman = friedmanchisquare(c_index_model_1,
                                                            c_index_model_2, c_index_model_3, c_index_model_4)
        friedman_p_values[tumor] = p_value_friedman

# Visualizza i p-value per ogni tumore
# print(f"Results for {model1} VS {model2}:")
for tumor in tumors:
    print(f"\nTumor: {tumor}")
    # print(f"Wilcoxon ({model1} vs {model2}): {wilcoxon_p_values[f'{model1}_vs_{model2}'][tumor]:.4f}")
    # print(f"Wilcoxon ({model1} vs {model3}): {wilcoxon_p_values[f'{model1}_vs_{model3}'][tumor]:.4f}")
    # print(f"Wilcoxon ({model1} vs {model4}): {wilcoxon_p_values[f'{model1}_vs_{model4}'][tumor]:.4f}")
    print(f"T-test ({model1} vs {model2}): {ttest_p_values[f'{model1}_vs_{model2}'][tumor]:.4f}")
    print(f"T-test ({model1} vs {model3}): {ttest_p_values[f'{model1}_vs_{model3}'][tumor]:.4f}")
    print(f"T-test ({model1} vs {model4}): {ttest_p_values[f'{model1}_vs_{model4}'][tumor]:.4f}")
    # print(f"Friedman test (all models): {friedman_p_values[tumor]:.4f}")


Processing tumor: BLCA
Processing tumor: COAD
Processing tumor: BRCA
Processing tumor: HNSC
Processing tumor: KIRC
Processing tumor: KIRP
Processing tumor: LUAD
Processing tumor: OV
Processing tumor: STAD
Processing tumor: LUSC

Tumor: BLCA
T-test (custom_multimodal vs HGCN): 0.1561
T-test (custom_multimodal vs MOTCat): 0.3916
T-test (custom_multimodal vs Survpath): 0.0947

Tumor: COAD
T-test (custom_multimodal vs HGCN): 0.9412
T-test (custom_multimodal vs MOTCat): 0.0459
T-test (custom_multimodal vs Survpath): 0.0824

Tumor: BRCA
T-test (custom_multimodal vs HGCN): 0.0117
T-test (custom_multimodal vs MOTCat): 0.1875
T-test (custom_multimodal vs Survpath): 0.7288

Tumor: HNSC
T-test (custom_multimodal vs HGCN): 0.1400
T-test (custom_multimodal vs MOTCat): 0.2037
T-test (custom_multimodal vs Survpath): 0.4396

Tumor: KIRC
T-test (custom_multimodal vs HGCN): 0.1310
T-test (custom_multimodal vs MOTCat): 0.7878
T-test (custom_multimodal vs Survpath): 0.0741

Tumor: KIRP
T-test (custom_mult

In [6]:
import pandas as pd
from scipy.stats import wilcoxon, friedmanchisquare
from scipy.stats import ttest_rel
mod='missing_all_60'
# Carica i due file CSV per i due modelli (ad esempio model_1.csv e model_2.csv)
df1 = pd.read_csv(f'/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/OUR_MODEL_{mod}_results_OS_TCGA.csv')
df2 = pd.read_csv(f'/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/HGCN_{mod}.csv')
model1 = 'custom_multimodal'
model2 = 'HGCN'
# model3='MOTCat'
# model4='Survpath'
filtered_df1 = df1[df1['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']]
# filtered_df3 = df3[df3['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']].rename(columns={'c-index_mean': 'c-index_mean_model_3'})
# filtered_df4 = df4[df4['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']].rename(columns={'c-index_mean': 'c-index_mean_model_4'})
df = pd.merge(filtered_df1,df2[['dataset_name', 'seed', 'c-index_mean']], on=['dataset_name', 'seed'],suffixes=('_model_1', '_model_2'))
# df= pd.merge(df,
#               filtered_df3[['dataset_name', 'seed', 'c-index_mean_model_3']],on=['dataset_name', 'seed'])
# df= pd.merge(df,filtered_df4[['dataset_name', 'seed', 'c-index_mean_model_4']], on=['dataset_name', 'seed'])
# Lista dei tumori unici nel dataset
tumors = df['dataset_name'].unique()
ttest_p_values = {
    f"{model1}_vs_{model2}": {},
    # f"{model1}_vs_{model3}": {},
    # f"{model1}_vs_{model4}": {}
}
wilcoxon_p_values = {
    f"{model1}_vs_{model2}": {},
    # f"{model1}_vs_{model3}": {},
    # f"{model1}_vs_{model4}": {}
}
friedman_p_values = {}
# Itera attraverso ogni tumore
for tumor in tumors:
    # print('Processing tumor:', tumor)
    # Filtra i dati per il tumore corrente
    tumor_data = df[df['dataset_name'] == tumor]
    
    # Assicurati di avere 3 valori di c-index per ogni seed
    if len(tumor_data) == 3:
        # Estrai i valori di c-index per i 2 modelli (model 1 e model 2) per i 3 seeds
        c_index_model_1 = tumor_data['c-index_mean_model_1'].values
        c_index_model_2 = tumor_data['c-index_mean_model_2'].values
        # c_index_model_3 = tumor_data['c-index_mean_model_3'].values
        # c_index_model_4 = tumor_data['c-index_mean_model_4'].values
        # Applica il test di Wilcoxon (test appaiato tra i 2 modelli)
        
        # Wilcoxon per ogni confronto con model_1
        wilcoxon_p_values[f"{model1}_vs_{model2}"][tumor] = wilcoxon(c_index_model_1, c_index_model_2).pvalue
        # wilcoxon_p_values[f"{model1}_vs_{model3}"][tumor] = wilcoxon(c_index_model_1, c_index_model_3).pvalue
        # wilcoxon_p_values[f"{model1}_vs_{model4}"][tumor] = wilcoxon(c_index_model_1, c_index_model_4).pvalue
        ttest_p_values[f"{model1}_vs_{model2}"][tumor] = ttest_rel(c_index_model_1, c_index_model_2).pvalue
        
        # # Applica il test di Friedman (per confrontare tutti i 3 seed)
        # stat_friedman, p_value_friedman = friedmanchisquare(c_index_model_1,
        #                                                     c_index_model_2, c_index_model_3, c_index_model_4)
        # friedman_p_values[tumor] = p_value_friedman

# Visualizza i p-value per ogni tumore
# print(f"Results for {model1} VS {model2}:")
for tumor in tumors:
    print(f"\nTumor: {tumor}")
    # print(f"Wilcoxon ({model1} vs {model2}): {wilcoxon_p_values[f'{model1}_vs_{model2}'][tumor]:.4f}")
    print(f"T-test ({model1} vs {model2}): {ttest_p_values[f'{model1}_vs_{model2}'][tumor]:.4f}")
    # print(f"Wilcoxon ({model1} vs {model3}): {wilcoxon_p_values[f'{model1}_vs_{model3}'][tumor]:.4f}")
    # print(f"Wilcoxon ({model1} vs {model4}): {wilcoxon_p_values[f'{model1}_vs_{model4}'][tumor]:.4f}")
    # print(f"Friedman test (all models): {friedman_p_values[tumor]:.4f}")



Tumor: BLCA
T-test (custom_multimodal vs HGCN): 0.7492

Tumor: HNSC
T-test (custom_multimodal vs HGCN): 0.1021

Tumor: BRCA
T-test (custom_multimodal vs HGCN): 0.1361

Tumor: LUAD
T-test (custom_multimodal vs HGCN): 0.0027

Tumor: LUSC
T-test (custom_multimodal vs HGCN): 0.0104

Tumor: STAD
T-test (custom_multimodal vs HGCN): 0.0219

Tumor: COAD
T-test (custom_multimodal vs HGCN): 0.1029

Tumor: KIRC
T-test (custom_multimodal vs HGCN): 0.0014

Tumor: OV
T-test (custom_multimodal vs HGCN): 0.0183

Tumor: KIRP
T-test (custom_multimodal vs HGCN): 0.8174


In [7]:
import pandas as pd
from scipy.stats import wilcoxon, friedmanchisquare
from scipy.stats import ttest_rel
mod='missing_all_30'
# Carica i due file CSV per i due modelli (ad esempio model_1.csv e model_2.csv)
df1 = pd.read_csv(f'/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/OUR_MODEL_{mod}_results_OS_TCGA.csv')
df2 = pd.read_csv(f'/work/H2020DeciderFicarra/D2_4/Development/MultimodalDecider/csv_results/HGCN_{mod}.csv')
model1 = 'custom_multimodal'
model2 = 'HGCN'
# model3='MOTCat'
# model4='Survpath'
filtered_df1 = df1[df1['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']]
# filtered_df3 = df3[df3['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']].rename(columns={'c-index_mean': 'c-index_mean_model_3'})
# filtered_df4 = df4[df4['dataset_name'].isin(df2['dataset_name'])][['dataset_name', 'seed', 'c-index_mean']].rename(columns={'c-index_mean': 'c-index_mean_model_4'})
df = pd.merge(filtered_df1,df2[['dataset_name', 'seed', 'c-index_mean']], on=['dataset_name', 'seed'],suffixes=('_model_1', '_model_2'))
# df= pd.merge(df,
#               filtered_df3[['dataset_name', 'seed', 'c-index_mean_model_3']],on=['dataset_name', 'seed'])
# df= pd.merge(df,filtered_df4[['dataset_name', 'seed', 'c-index_mean_model_4']], on=['dataset_name', 'seed'])
# Lista dei tumori unici nel dataset
tumors = df['dataset_name'].unique()
ttest_p_values = {
    f"{model1}_vs_{model2}": {},
    # f"{model1}_vs_{model3}": {},
    # f"{model1}_vs_{model4}": {}
}
wilcoxon_p_values = {
    f"{model1}_vs_{model2}": {},
    # f"{model1}_vs_{model3}": {},
    # f"{model1}_vs_{model4}": {}
}
friedman_p_values = {}
# Itera attraverso ogni tumore
print(' missing_all_30')
for tumor in tumors:
    # print('Processing tumor:', tumor)
    # Filtra i dati per il tumore corrente
    tumor_data = df[df['dataset_name'] == tumor]
    
    # Assicurati di avere 3 valori di c-index per ogni seed
    if len(tumor_data) == 3:
        # Estrai i valori di c-index per i 2 modelli (model 1 e model 2) per i 3 seeds
        c_index_model_1 = tumor_data['c-index_mean_model_1'].values
        c_index_model_2 = tumor_data['c-index_mean_model_2'].values
        # c_index_model_3 = tumor_data['c-index_mean_model_3'].values
        # c_index_model_4 = tumor_data['c-index_mean_model_4'].values
        # Applica il test di Wilcoxon (test appaiato tra i 2 modelli)
        
        # Wilcoxon per ogni confronto con model_1
        wilcoxon_p_values[f"{model1}_vs_{model2}"][tumor] = wilcoxon(c_index_model_1, c_index_model_2).pvalue
        # wilcoxon_p_values[f"{model1}_vs_{model3}"][tumor] = wilcoxon(c_index_model_1, c_index_model_3).pvalue
        # wilcoxon_p_values[f"{model1}_vs_{model4}"][tumor] = wilcoxon(c_index_model_1, c_index_model_4).pvalue

        ttest_p_values[f"{model1}_vs_{model2}"][tumor] = ttest_rel(c_index_model_1, c_index_model_2).pvalue
        # # Applica il test di Friedman (per confrontare tutti i 3 seed)
        # stat_friedman, p_value_friedman = friedmanchisquare(c_index_model_1,
        #                                                     c_index_model_2, c_index_model_3, c_index_model_4)
        # friedman_p_values[tumor] = p_value_friedman

# Visualizza i p-value per ogni tumore
# print(f"Results for {model1} VS {model2}:")
for tumor in tumors:
    print(f"\nTumor: {tumor}")
    # print(f"Wilcoxon ({model1} vs {model2}): {wilcoxon_p_values[f'{model1}_vs_{model2}'][tumor]:.4f}")
    print(f"T-test ({model1} vs {model2}): {ttest_p_values[f'{model1}_vs_{model2}'][tumor]:.4f}")
    # print(f"Wilcoxon ({model1} vs {model3}): {wilcoxon_p_values[f'{model1}_vs_{model3}'][tumor]:.4f}")
    # print(f"Wilcoxon ({model1} vs {model4}): {wilcoxon_p_values[f'{model1}_vs_{model4}'][tumor]:.4f}")
    # print(f"Friedman test (all models): {friedman_p_values[tumor]:.4f}")


 missing_all_30

Tumor: BLCA
T-test (custom_multimodal vs HGCN): 0.9380

Tumor: HNSC
T-test (custom_multimodal vs HGCN): 0.0146

Tumor: BRCA
T-test (custom_multimodal vs HGCN): 0.5850

Tumor: LUAD
T-test (custom_multimodal vs HGCN): 0.6881

Tumor: LUSC
T-test (custom_multimodal vs HGCN): 0.0369

Tumor: STAD
T-test (custom_multimodal vs HGCN): 0.0061

Tumor: COAD
T-test (custom_multimodal vs HGCN): 0.2834

Tumor: KIRC
T-test (custom_multimodal vs HGCN): 0.0140

Tumor: OV
T-test (custom_multimodal vs HGCN): 0.0059

Tumor: KIRP
T-test (custom_multimodal vs HGCN): 0.0153
